In [1]:
import ee
import geemap

In [2]:
ee.Initialize()

### Define premise parameter

In [3]:
# Define the time-range
# just get one_range from every different sensors
year_range = [(f'{i}-01-01',f'{i+2}-12-31') for i in range(1990,2020,3)][:2]

In [4]:
year_range

[('1990-01-01', '1992-12-31'), ('1993-01-01', '1995-12-31')]

##### Adjust below code to your research area

In [5]:
Target_area = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
                .filterMetadata('NAME1','equals','西北')

In [6]:
# define the asset path for exporting
asset_path = 'users/wensomone666/China_built_up/01_Data_preparation/03_Normalized_img'

### [Mean_NDVI, Mean_NDBI, Mean_EVI] calcualtion according to year_range

In [7]:
# use if condition to find out the right Landsat img formulation
# here use Landsat Raw data

Normalized_img = []
            
# reverse the list so we can export the recent img first
for span in year_range:
    
    # find the end year to determine the range_name
    start_year = int(span[0].split('-')[0])
    end_year   = int(span[1].split('-')[0])
    range_name = f'{start_year}_{end_year}'

    if end_year <= 2010:
        Landsat_img = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")
        ND_formula = {'NDVI':['B4','B3'],
                          'NDBI':['B5','B4'],
                          'EVI':"2.5 * ((b('B4')-b('B3'))*1.0 / (b('B4')*1.0 + 6.0 * b('B3') - 7.5 * b('B1') + 1.0))"}

    elif end_year <= 2013:
        Landsat_img = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA")
        ND_formula = {'NDVI':['B4','B3'],
                      'NDBI':['B5','B4'],
                      'EVI':"2.5 * ((b('B4')-b('B3'))*1.0 / (b('B4')*1.0 + 6.0 * b('B3') - 7.5 * b('B1') + 1.0))"}

    else:
        Landsat_img = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
        ND_formula = {'NDVI':['B5','B4'],
                          'NDBI':['B6','B5'],
                          'EVI':"2.5 * ((b('B5')-b('B4'))*1.0 / (b('B5')*1.0 + 6.0 * b('B4') - 7.5 * b('B2') + 1.0))"}
    
    # compute the normalized difference imgs
    # multiply the NDVI with 100 and change it 
    # into a int-img to reduce space
    NDVI_mean = Landsat_img\
                            .filterBounds(Target_area)\
                            .filterDate(span[0],span[1])\
                            .mean()\
                            .normalizedDifference(ND_formula['NDVI'])\
                            .multiply(100).toInt8()
    
    NDBI_mean = Landsat_img\
                            .filterBounds(Target_area)\
                            .filterDate(span[0],span[1])\
                            .mean()\
                            .normalizedDifference(ND_formula['NDBI'])\
                            .multiply(100).toInt8()
    
    EVI_mean  = Landsat_img\
                            .filterBounds(Target_area)\
                            .filterDate(span[0],span[1])\
                            .mean()\
                            .expression(ND_formula['EVI'])\
                            .multiply(100).toInt8()
    
    
    # add the range_name and landsat img to the list
    Normalized_img.append((range_name,[NDVI_mean,NDBI_mean,EVI_mean]))

### Export the Normalized img

In [8]:
for span_range,img in Normalized_img:
    
    # define the name              
    name = f'NDVI_NDBI_EVI_{span_range}'
    
    # cast img to ee.Image
    img = ee.Image(img).rename(['NDVI','NDBI','EVI'])
    
    # export
    task = ee.batch.Export.image.toAsset(   image           = img,
                                            description     = name,
                                            assetId         = f'{asset_path}/{name}',
                                            region          = Target_area.geometry(),
                                            scale           = 30,
                                            maxPixels       = int(1e13)
                                        )

    task.start()

    # print out the process
    print(f'Exporting Mean-Normalized-img of {name}')


Exporting Mean-Normalized-img of NDVI_NDBI_EVI_1990_1992
Exporting Mean-Normalized-img of NDVI_NDBI_EVI_1993_1995


##### Add img to map

In [9]:
one_img = img

In [11]:
# show the map for validation
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.setCenter(120.3482, 33.0212,8)

# add the img
Map.addLayer(one_img,{'min':-10,'max':35},'NDVI_NDBI_EVI')


# show the img
Map